# **지능형 IoT 응용**
#### 한국기술교육대학교 컴퓨터공학부 스마트 IoT 트랙


---

# 08. 마이크로컨트롤러용 인체 감지 애플리케이션

---

### Acknowledgement


이 자료는 다음 서적의 내용을 바탕으로 작성되었음
- 초소형 머신러닝 TinyML, 피트 워든, 대니얼 시투나야케 지음, 맹윤호, 임지순 옮김, 한빛미디어
  - 9장

---

### 비전 인식 모델

- CNN의 발전으로 기계가 '볼 수 있게' 하는 프로그램을 만들기가 쉬워짐
- 현재 비전 모델은 다양한 작업에 사용되고 있음
  - 자율주행 차량은 비전을 사용하여 도로에서 위험을 발견함
  - 공장 로봇은 카메라를 사용하여 부품 결함을 포착함
  - 의료 영상을 통해 질병을 진단함
  - 스마트폰에서 사진을 찍을 때 얼굴을 인식해 초점을 맞출 수 있음
- 비전 인식이 가능한 기계는 기존에 기계로 할 수 없었던 많은 작업을 자동화할 수 있음

- 하지만 사람들은 언제 어디서나 비전 인식이 가능하여 자신의 행동이 기록되거나 클라우드로 스트리밍되어 기계에 읽히는 것을 반기지 않을 것임
  - 보안 시스템, 사람이 있는지 없는지 인식하는 난방기, 방에 사람이 없을 때 자동으로 꺼지는 텔레비전 같은 내장 카메라가 달린 가전 제품을 사용할 때 중요한 것은 프라이버시
  - 항상 인터넷에 연결된 장치에 카메라까지 내장되어 있다면 보안 측면에서 소비자에게 문제를 야기할 수 있음


### 마이크로컨트롤러 기반 비전 인식

- 인터넷에 연결되지 않은 작은 마이크로컨트롤러를 사용하여 근처에 사람이 있는지 여부를 감지할 수 있다면 프라이버시를 포기하지 않고도 위에서 언급한 것 같은 스마트 기기의 장점을 취할 수 있음

- 마이크로컨트롤러 기반 비전 시스템은 저전력 특성 덕분에 소형 배터리로 장기간 구동될 수도 있음

- 특정 물체가 보이면 1, 그렇지 않으면 0을 출력하며 카메라가 수집한 이미지 데이터를 외부에 공유하지 않는 독립형 전자 부품 형태의 비전 센서도 만들 수도 있음
  - 이러한 센서는 스마트 홈 시스템에서 개인 차량에 이르기까지 다양한 제품에 내장될 수 있음
  - 차가 뒤에 있을 때 플래시를 켜는 자전거
  - 집에 사람이 있는지 인식하는 에어컨

## 8.1 인체 감지 애플리케이션

- 인체 감지 모델을 기반으로 사람이 있는지 알아내어 LED를 제어하는 임베디드 애플리케이션
  - 인체 감지 모델: 카메라로 캡처한 이미지에 사람이 있을 때 이를 인식하는 모델
  - 사람 있음 vs. 없음으로 binary classification(이진 분류)을 수행하는 모델임

- 애플리케이션 예제 코드
  - TinyML 번역서의 한글 소스코드 저장소의 예제 코드
  - https://github.com/yunho0130/tensorflow-lite/tree/master/tensorflow/lite/micro/examples/person_detection





- 일반적인 머신러닝 애플리케이션 동작
  - 입력을 받는다
  - 입력을 전처리하여 모델에 공급하기에 적합한 특징을 추출한다
  - 처리된 입력의 특징을 이용하여 모델 추론을 실행한다
  - 모델의 출력을 후처리한다
  - 결과 정보를 사용하여 동작을 수행한다

- 인체 감지 애플리케이션은 이미지 데이터를 입력을 사용
  - 이전에 봤던 호출어 감지 애플리케이션은 오디오 데이터를 입력으로 사용하고 모델 입력을 만들기 위해 오디오 데이터에서 특징(스펙트로그램)을 추출하는 과정이 있었음
  - 하지만 픽셀 값의 배열로 표현되는 이미지 데이터는 이 형식 그대로 모델의 입력이 되므로 모델에 데이터를 공급하기 전 많은 전처리가 필요 없음
  - 애플리케이션 동작은 카메라에서 데이터의 스냅샷을 찍어 모델로 공급하고, 출력 클래스를 결정하고, 간단한 방법으로 결과를 표시하는 것이 전부여서 비교적 간단함
  - 인체 감지 모델은 추론을 실행하는 데 더 오래 걸리기 때문에 모델 출력값의 평균을 구하지 않아도 되는 형태여서 구조가 호출어 감지 애플리케이션에 비해 더 간단해짐




## 8.2 애플리케이션 아키텍처


<img src="./08.인체_감지_애플리케이션/08-1.architecture.png" width="90%" height="90%">
<center>인체 감지 애플리케이션 아키텍처</center>

### 애플리케이션 구성 요소

- 메인 루프
  - 다른 애플리케이션 예제와 마찬가지로 인체 감지 애플리케이션도 연속 루프에서 실행됨
  - 모델이 훨씬 크고 복잡하기 때문에 추론을 실행하는데 시간이 더 걸림
  - 장치에 따라 초당 몇 개의 추론이 아니라 수 초마다 한 개의 추론을 수행할 수도 있음

- 이미지 추출기
  - 카메라에서 이미지 데이터를 캡처하여 입력 텐서에 씀
  - 이미지를 캡처하는 방법은 장치마다 다르므로 이 구성 요소를 재정의하고 커스텀 버전을 만들 수 있음

- 텐서플로우 라이트 인터프리터
  - 텐서플로우 라이트 모델을 실행하여 입력 이미지를 확률 세트로 변환함

- 모델
  - 인체 유무를 감지하도록 학습됨
  - 인터프리터에 의해 실행됨
  - 용량이 250KB 정도

- 감지 응답기
  - 모델에서 출력된 확률 데이터로 감지 여부를 결정하고 이를 장치의 출력 기능으로 표시함
  - 이 부분은 장치에 따라 커스텀 버전을 사용할 수 있음
  - 예제 코드에서는 LED를 켜는 형태로 출력

### 예제 모델 소개

- 인체 감지 모델: Visual Wake Words 데이터셋을 통해 훈련된 CNN 기반 모델
  - 구체적으로는 MobileNet v1 아키텍처를 사용
  - MobileNet은 스마트폰과 같은 모바일 장치에서도 이미지 분류를 수행할 수 있도록 연산량과 파라미터 수를 대폭 줄인 신경망
  - convolution 연산을 수행하는데 depthwise separable convolution 연산 방식으로 변경하여 정확도를 유지하면서도 훨씬 효율적인 방식으로 convolution 연산 수행 가능 (참고: https://yunmorning.tistory.com/58 / https://housekdk.gitbook.io/ml/ml/deployment/mobilenet)

- Depthwise separable convolution
  - Depthwise convolution 연산 후 pointwise convolution(1x1 conv) 연산을 수행

 <center><img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*JwCJCgN2UreEn3U1nwVj8Q.png"></center>
 <center>(이미지 출처: https://medium.com/@zurister/depth-wise-convolution-and-depth-wise-separable-convolution-37346565d4ec)</center>







 - MobileNet 모델 아키텍처

 <center><img src="https://blog.kakaocdn.net/dn/mJMrv/btqEjB8MDVx/8F1FjJ6TpxfW8ZALMIzzq0/img.png"></center>
 <center>(이미지 출처: https://yeomko.tistory.com/45)</center>


- 모델 용량은 250KB로 호출어 감지 모델보다 훨씬 큼. 용량이 커서 더 많은 메모리를 차지할 뿐 아니라 추론을 실행하는데 시간이 더 오래 걸림

- Visual Wake Words 데이터셋
  - 데이터셋은 11만 5000개의 이미지로 구성
  - 각 이미지는 사람이 포함되어 있는지 여부를 나타내는 레이블이 매겨짐
  - https://github.com/Mxbonn/visualwakewords
  - 관련 arXiv 논문 링크: https://arxiv.org/abs/1906.05721

- 모델의 입력
  - 96 x 96 픽셀 그레이스케일 이미지
  - 각 이미지는 (96, 96, 1)의 형태를 가지는 3D 텐서
  - 마지막 차원은 단일 픽셀을 나타내는 8비트 값 - 0(완전 검정)에서 255(완전 흰색)까지 픽셀의 음영을 지정
  - 96 x 96 픽셀 이미지는 작은 해상도로 보이지만 각 이미지에서 사람을 감지하기에는 충분

- 모델의 출력
  - 사람 있음, 사람 없음의 두 가지 클래스를 분류
  - 두 가지 확률을 출력하며 확률의 값은 0에서 255 사이의 범위의 8비트 정수로 표현

## 8.3 테스트 코드



#### 주요 코드

- person_detection_test.cc: 단일 이미지를 나타내는 배열에서 추론을 실행하는 방법
- image_provider_test.cc: 이미지 추출기를 사용하여 이미지를 캡처하는 방법
- detection_responder_test.cc: 감지 응답기를 사용하여 감지 결과를 출력하는 방법

##### [github 저장소]
- https://github.com/yunho0130/tensorflow-lite/tree/master/tensorflow/lite/micro/examples/person_detection

### 8.3.1 기본 흐름 (person_detection_test.cc)

- 모델 로드, 인터프리터 설정, 텐서 할당, 인터프리터 실행 등의 기본 동작은 이전 예제와 비슷함




```cpp
#include "tensorflow/lite/c/common.h"
#include "tensorflow/lite/micro/examples/person_detection/model_settings.h"
#include "tensorflow/lite/micro/examples/person_detection/no_person_image_data.h"
#include "tensorflow/lite/micro/examples/person_detection/person_detect_model_data.h"
#include "tensorflow/lite/micro/examples/person_detection/person_image_data.h"
#include "tensorflow/lite/micro/kernels/micro_ops.h"
#include "tensorflow/lite/micro/micro_error_reporter.h"
#include "tensorflow/lite/micro/micro_interpreter.h"
#include "tensorflow/lite/micro/micro_mutable_op_resolver.h"
#include "tensorflow/lite/micro/testing/micro_test.h"
#include "tensorflow/lite/schema/schema_generated.h"
#include "tensorflow/lite/version.h"
```

- tensorflow/lite/micro/examples/person_detection/no_person_image_data.h
  - 테스트 용으로 사용하는 사람이 없는 이미지 데이터
  - https://raw.githubusercontent.com/tensorflow/tflite-micro/main/tensorflow/lite/micro/examples/person_detection/testdata/no_person.bmp

- tensorflow/lite/micro/examples/person_detection/person_image_data.h
  - 테스트 용으로 사용하는 사람이 있는 이미지 데이터
  - https://raw.githubusercontent.com/tensorflow/tflite-micro/main/tensorflow/lite/micro/examples/person_detection/testdata/person.bmp





```cpp
// Create an area of memory to use for input, output, and intermediate arrays.
constexpr int tensor_arena_size = 73 * 1024;
uint8_t tensor_arena[tensor_arena_size];
```

- 모델에 적합한 크기의 텐서 아레나 생성
  - 입력, 출력, 중간 데이터 텐서를 저장하는 메모리 공간

```cpp
  // Set up logging.
  tflite::MicroErrorReporter micro_error_reporter;
  tflite::ErrorReporter* error_reporter = &micro_error_reporter;

  // Map the model into a usable data structure. This doesn't involve any
  // copying or parsing, it's a very lightweight operation.
  const tflite::Model* model = ::tflite::GetModel(g_person_detect_model_data);
  if (model->version() != TFLITE_SCHEMA_VERSION) {
    TF_LITE_REPORT_ERROR(error_reporter,
                         "Model provided is schema version %d not equal "
                         "to supported version %d.\n",
                         model->version(), TFLITE_SCHEMA_VERSION);
  }
```

- 로깅을 위한 ErrorReporter 객체 생성
- Model 객체 생성
  - 모델 데이터로는 person_detect_model_data.h에 정의된 g_person_detect_model_data 배열 이용



```cpp
  // Pull in only the operation implementations we need.
  // This relies on a complete list of all the ops needed by this graph.
  // An easier approach is to just use the AllOpsResolver, but this will
  // incur some penalty in code space for op implementations that are not
  // needed by this graph.
  //
  // tflite::ops::micro::AllOpsResolver resolver;
  tflite::MicroOpResolver<3> micro_op_resolver;
  micro_op_resolver.AddBuiltin(
      tflite::BuiltinOperator_DEPTHWISE_CONV_2D,
      tflite::ops::micro::Register_DEPTHWISE_CONV_2D());
  micro_op_resolver.AddBuiltin(tflite::BuiltinOperator_CONV_2D,
                               tflite::ops::micro::Register_CONV_2D());
  micro_op_resolver.AddBuiltin(tflite::BuiltinOperator_AVERAGE_POOL_2D,
                               tflite::ops::micro::Register_AVERAGE_POOL_2D());
```

- MicroOpResolver를 사용하여 필요한 Op을 추가

```cpp
  // Build an interpreter to run the model with.
  tflite::MicroInterpreter interpreter(model, micro_op_resolver, tensor_arena,
                                       tensor_arena_size, error_reporter);
  interpreter.AllocateTensors();
```

- 인터프리터 객체 생성
  - 위에서 생성한 Model 객체, MicroOpResolver 객체, Tensor Arena, ErrorReporter 객체 이용
- 텐서 할당

```cpp
  // Get information about the memory area to use for the model's input.
  TfLiteTensor* input = interpreter.input(0);

  // Make sure the input has the properties we expect.
  TF_LITE_MICRO_EXPECT_NE(nullptr, input);
  TF_LITE_MICRO_EXPECT_EQ(4, input->dims->size);
  TF_LITE_MICRO_EXPECT_EQ(1, input->dims->data[0]);
  TF_LITE_MICRO_EXPECT_EQ(kNumRows, input->dims->data[1]);
  TF_LITE_MICRO_EXPECT_EQ(kNumCols, input->dims->data[2]);
  TF_LITE_MICRO_EXPECT_EQ(kNumChannels, input->dims->data[3]);
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteUInt8, input->type);
```

- 인터프리터의 입력 텐서에 대한 포인터를 이용해 입력 텐서가 올바른 형태인지 확인
  - null 포인터가 아니어야 함
  - 입력 텐서의 차원은 4
  - 첫번째 차원(data[0])은 단일 요소를 포함하는 래퍼
  - 두번째 차원(data[1])은 이미지 픽셀의 행 (96)
  - 세번째 차원(data[2])은 이미지 픽셀의 열 (96)
  - 네번째 차원(data[3])은 이미지 픽셀의 채널 (1)
  - 데이터 타입은 8비트 integer(kTfLiteInt8)
  - kNumRows, kNumCols, kNumChannels는 model_settings.h에 정의된 상수 (https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/person_detection/model_settings.h)

- model_settings.h
```cpp
// All of these values are derived from the values used during model training,
// if you change your model you'll need to update these constants.
constexpr int kNumCols = 96;
constexpr int kNumRows = 96;
constexpr int kNumChannels = 1;
constexpr int kMaxImageSize = kNumCols * kNumRows * kNumChannels;
constexpr int kCategoryCount = 3;
constexpr int kPersonIndex = 1;
constexpr int kNotAPersonIndex = 2;
extern const char* kCategoryLabels[kCategoryCount];
```


```cpp
  // Copy an image with a person into the memory area used for the input.
  const uint8_t* person_data = g_person_data;
  for (int i = 0; i < input->bytes; ++i) {
    input->data.uint8[i] = person_data[i];
  }
```

- 테스트 추론을 위해 사람이 있는 이미지 데이터(g_person_data 배열)를 입력 텐서(iniput->data)에 복사
  - g_person_data는 person_image_data.h에 정의됨


```cpp
  // Run the model on this input and make sure it succeeds.
  TfLiteStatus invoke_status = interpreter.Invoke();
  if (invoke_status != kTfLiteOk) {
    TF_LITE_REPORT_ERROR(error_reporter, "Invoke failed\n");
  }
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteOk, invoke_status);
```

- 인터프리터를 실행하고 에러가 없는지 확인


```cpp
  // Get the output from the model, and make sure it's the expected size and
  // type.
  TfLiteTensor* output = interpreter.output(0);
  TF_LITE_MICRO_EXPECT_EQ(4, output->dims->size);
  TF_LITE_MICRO_EXPECT_EQ(1, output->dims->data[0]);
  TF_LITE_MICRO_EXPECT_EQ(1, output->dims->data[1]);
  TF_LITE_MICRO_EXPECT_EQ(1, output->dims->data[2]);
  TF_LITE_MICRO_EXPECT_EQ(kCategoryCount, output->dims->data[3]);
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteUInt8, output->type);
```

- 인터프리터의 출력 텐서에 대한 포인터를 이용해 출력 텐서가 올바를 형태인지 확인
  - 출력 텐서의 차원은 4
  - 첫번째, 두번째, 세번째 차원(data[0], data[1], data[2])은 단일 요소를 포함하는 래퍼
  - 네번째 차원(data[3])은 모델 추론 결과를 저장. 2가지 클래스(notperson, person) 각각의 확률
  - 데이터 타입은 8비트 integer


```cpp
  // Make sure that the expected "Person" score is higher than the other class.
  uint8_t person_score = output->data.uint8[kPersonIndex];
  uint8_t no_person_score = output->data.uint8[kNotAPersonIndex];
  TF_LITE_REPORT_ERROR(error_reporter,
                       "person data.  person score: %d, no person score: %d\n",
                       person_score, no_person_score);
  TF_LITE_MICRO_EXPECT_GT(person_score, no_person_score);
```

- '사람 있음'과 '사람 없음' 점수를 읽어서 ErrorReporter를 이용하여 기록
- '사람 있음' 점수(person_score)가 '사람 없음' 점수(no_person_score)보다 큰지 확인

```cpp
  // Now test with a different input, from an image without a person.
  const uint8_t* no_person_data = g_no_person_data;
  for (int i = 0; i < input->bytes; ++i) {
    input->data.uint8[i] = no_person_data[i];
  }
```

- 테스트 추론을 위해 사람이 없는 이미지 데이터(g_no_person_data)를 입력 텐서(iniput->data)에 복사
  - g_no_person_data는 no_person_image_data.h에 정의됨

```cpp
  // Run the model on this "No Person" input.
  invoke_status = interpreter.Invoke();
  if (invoke_status != kTfLiteOk) {
    TF_LITE_REPORT_ERROR(error_reporter, "Invoke failed\n");
  }
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteOk, invoke_status);
```

- 인터프리터를 실행하고 에러가 없는지 확인

```cpp
  // Get the output from the model, and make sure it's the expected size and
  // type.
  output = interpreter.output(0);
  TF_LITE_MICRO_EXPECT_EQ(4, output->dims->size);
  TF_LITE_MICRO_EXPECT_EQ(1, output->dims->data[0]);
  TF_LITE_MICRO_EXPECT_EQ(1, output->dims->data[1]);
  TF_LITE_MICRO_EXPECT_EQ(1, output->dims->data[2]);
  TF_LITE_MICRO_EXPECT_EQ(kCategoryCount, output->dims->data[3]);
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteUInt8, output->type);
```

- 인터프리터의 출력 텐서에 대한 포인터를 이용해 출력 텐서가 올바를 형태인지 확인
  - 출력 텐서의 차원은 4
  - 첫번째, 두번째, 세번째 차원(data[0], data[1], data[2])은 단일 요소를 포함하는 래퍼
  - 네번째 차원(data[3])은 모델 추론 결과를 저장. 2가지 클래스(notperson, person) 각각의 확률
  - 데이터 타입은 8비트 integer


```cpp
  // Make sure that the expected "No Person" score is higher.
  person_score = output->data.uint8[kPersonIndex];
  no_person_score = output->data.uint8[kNotAPersonIndex];
  TF_LITE_REPORT_ERROR(
      error_reporter,
      "no person data.  person score: %d, no person score: %d\n", person_score,
      no_person_score);
  TF_LITE_MICRO_EXPECT_GT(no_person_score, person_score);
```

- '사람 있음'과 '사람 없음' 점수를 읽어서 ErrorReporter를 이용하여 기록
- '사람 없음' 점수(no_person_score)가 '사람 있음' 점수(person_score)보다 큰지 확인

#### 테스트 실행


In [1]:
%cd /content
%rm -rf tensorflow
! git clone https://github.com/yunho0130/tensorflow-lite

/content
Cloning into 'tensorflow-lite'...
remote: Enumerating objects: 847199, done.
remote: Counting objects: 100% (28947/28947), done.
remote: Compressing objects: 100% (515/515), done.
remote: Total 847199 (delta 28720), reused 28432 (delta 28432), pack-reused 818252
Receiving objects: 100% (847199/847199), 595.89 MiB | 26.12 MiB/s, done.
Resolving deltas: 100% (678730/678730), done.
Updating files: 100% (19934/19934), done.


In [2]:
%cd /content/tensorflow-lite/

! make -f tensorflow/lite/micro/tools/make/Makefile test_person_detection_test

/content/tensorflow-lite
tensorflow/lite/micro/tools/make/download_and_extract.sh "https://github.com/google/gemmlowp/archive/719139ce755a0f31cbf1c37f7f98adcc7fc9f425.zip" "7e8191b24853d75de2af87622ad293ba" tensorflow/lite/micro/tools/make/downloads/gemmlowp  
downloading https://github.com/google/gemmlowp/archive/719139ce755a0f31cbf1c37f7f98adcc7fc9f425.zip
tensorflow/lite/micro/tools/make/download_and_extract.sh "https://storage.googleapis.com/mirror.tensorflow.org/github.com/google/flatbuffers/archive/v1.11.0.tar.gz" "02c64880acb89dbd57eebacfd67200d8" tensorflow/lite/micro/tools/make/downloads/flatbuffers  
downloading https://storage.googleapis.com/mirror.tensorflow.org/github.com/google/flatbuffers/archive/v1.11.0.tar.gz
tensorflow/lite/micro/tools/make/download_and_extract.sh "https://github.com/mborgerding/kissfft/archive/v130.zip" "438ba1fef5783cc5f5f201395cc477ca" tensorflow/lite/micro/tools/make/downloads/kissfft patch_kissfft 
downloading https://github.com/mborgerding/kissf

### 8.3.2 이미지 추출기 (image_provider.h / image_provider.cc)

- 카메라에서 캡처한 이미지 데이터를 가져와 모델의 입력 텐서에 쓰기 적합한 형식으로 반환
  - 장치마다 이미지 캡처를 위한 구현이 다르기 때문에 image_provider.h는 이미지 데이터 추출을 위한 인터페이스를 정의

- 이미지 추출기의 핵심은 GetImage() 함수

```cpp
// The reference implementation can have no platform-specific dependencies, so
// it just returns a static image. For real applications, you should
// ensure there's a specialized implementation that accesses hardware APIs.
TfLiteStatus GetImage(tflite::ErrorReporter* error_reporter, int image_width,
                      int image_height, int channels, uint8_t* image_data);
```

- 8비트 이미지 데이터 배열을 반환
- 5개의 매개변수
  - ErrorReporter 객체
  - 이미지 폭(width)
  - 이미지 높이(height)
  - 이미지 채널수
  - 이미지 데이터를 저장할 배열에 대한 포인터

 - image_provider.cc에는 더미 데이터를 반환하는 참조 코드가 구현되어 있음

```cpp
TfLiteStatus GetImage(tflite::ErrorReporter* error_reporter, int image_width,
                      int image_height, int channels, uint8_t* image_data) {
  for (int i = 0; i < image_width * image_height * channels; ++i) {
    image_data[i] = 0;
  }
  return kTfLiteOk;
}
```

- 이미지 데이터 배열에 모두 0을 채움


#### 이미지 추출기 테스트

- 이미지 추출기를 사용하는 방법
  - image_provider_test.cc의 테스트 확인

```cpp
TF_LITE_MICRO_TEST(TestImageProvider) {
  tflite::MicroErrorReporter micro_error_reporter;
  tflite::ErrorReporter* error_reporter = &micro_error_reporter;

  uint8_t image_data[kMaxImageSize];
    
  TfLiteStatus get_status =
      GetImage(error_reporter, kNumCols, kNumRows, kNumChannels, image_data);
    
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteOk, get_status);
  TF_LITE_MICRO_EXPECT_NE(image_data, nullptr);

  // Make sure we can read all of the returned memory locations.
  uint32_t total = 0;
  for (int i = 0; i < kMaxImageSize; ++i) {
    total += image_data[i];
  }
}
```

- ErrorReporter 객체 생성

- 이미지 데이터를 저장할 8비트 int 배열 생성 (image_data)
  - kMaxImageSize는 model_settings.h에서 정의
  - 96 * 96 * 1

- GetImage 함수 호출
  - kNumCols, kNumRows, kNumChannels도 model_settings.h에서 정의

- 함수 호출 반환값으로 에러 여부 확인

- image_data가 null 포인터가 아닌지 확인

- 마지막 for 루프에서 이미지 데이터 배열에서 값을 모두 읽을 수 있는지 확인

In [3]:
! make -f tensorflow/lite/micro/tools/make/Makefile test_image_provider_test

g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/person_detection/image_provider.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/person_detection/image_provider.o
g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/person_detection/image_provider_test.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/person_detection/image_provider_test.o
g++ -std=c++11 -DTF_LITE

### 8.3.3 감지 응답기 (detection_responder.h / detection_responder.cc)

- 감지 응답기 역할
  - 추론 결과에 따른 응답 실행
  - RespondToDetection() 함수

- 감지 응답기는 각 유형의 장치에 따라 재정의되도록 설계됨

```cpp
// Called every time the results of a person detection run are available. The
// `person_score` has the numerical confidence that the captured image contains
// a person, and `no_person_score` has the numerical confidence that the image
// does not contain a person. Typically if person_score > no person score, the
// image is considered to contain a person.  This threshold may be adjusted for
// particular applications.
void RespondToDetection(tflite::ErrorReporter* error_reporter,
                        uint8_t person_score, uint8_t no_person_score);
```

- RespondToDetection() 함수 정의

- 3개의 매개변수
  - ErrorReporter 객체
  - '사람 있음'의 점수
  - '사람 없음'의 점수

```cpp
// This dummy implementation writes person and no person scores to the error
// console. Real applications will want to take some custom action instead, and
// should implement their own versions of this function.
void RespondToDetection(tflite::ErrorReporter* error_reporter,
                        uint8_t person_score, uint8_t no_person_score) {
  TF_LITE_REPORT_ERROR(error_reporter, "person score:%d no person score %d",
                       person_score, no_person_score);
}
```

- detection_responder.cc에는 단순히 추론 결과 점수를 출력하도록 구현됨

#### 감지 응답기 테스트

- 테스트 코드
  - detection_responder_test.cc

```cpp
TF_LITE_MICRO_TEST(TestCallability) {
  tflite::MicroErrorReporter micro_error_reporter;
  tflite::ErrorReporter* error_reporter = &micro_error_reporter;

  // This will have external side-effects (like printing to the debug console
  // or lighting an LED) that are hard to observe, so the most we can do is
  // make sure the call doesn't crash.
  RespondToDetection(error_reporter, 100, 200);
  RespondToDetection(error_reporter, 200, 100);
}
```

- RespondToDetection() 함수를 2번 호출하는 테스트


In [4]:
! make -f tensorflow/lite/micro/tools/make/Makefile test_detection_responder_test

g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/person_detection/detection_responder.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/person_detection/detection_responder.o
g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/person_detection/detection_responder_test.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/person_detection/detection_responder_test.o
g++ 

## 8.4 인체 감지

#### 주요 코드
- main_functions.h / main_functions.cc
  - 프로그램의 핵심인 setup(), loop() 함수를 정의
  - person_detection_test.cc에서 본 것과 유사한 형태

- main.cc
  - 여기에 정의된 main() 함수에서 setup(), loop() 함수 호출

#### 초기화

```cpp
// Globals, used for compatibility with Arduino-style sketches.
namespace {
tflite::ErrorReporter* error_reporter = nullptr;
const tflite::Model* model = nullptr;
tflite::MicroInterpreter* interpreter = nullptr;
TfLiteTensor* input = nullptr;

// An area of memory to use for input, output, and intermediate arrays.
constexpr int kTensorArenaSize = 73 * 1024;
static uint8_t tensor_arena[kTensorArenaSize];
}  // namespace
```

- 전역 변수 선언
  - ErrorReporter, 모델, 인터프리터, 입력 텐서
  - 텐서 아레나

#### setup() 함수



```cpp
// The name of this function is important for Arduino compatibility.
void setup() {
  // Set up logging. Google style is to avoid globals or statics because of
  // lifetime uncertainty, but since this has a trivial destructor it's okay.
  // NOLINTNEXTLINE(runtime-global-variables)
  static tflite::MicroErrorReporter micro_error_reporter;
  error_reporter = &micro_error_reporter;

  // Map the model into a usable data structure. This doesn't involve any
  // copying or parsing, it's a very lightweight operation.
  model = tflite::GetModel(g_person_detect_model_data);
  if (model->version() != TFLITE_SCHEMA_VERSION) {
    TF_LITE_REPORT_ERROR(error_reporter,
                         "Model provided is schema version %d not equal "
                         "to supported version %d.",
                         model->version(), TFLITE_SCHEMA_VERSION);
    return;
  }
```

- ErrorReporter 객체 생성
- Model 객체 생성
  - 인체 감지 모델 데이터는 g_person_detect_model_data 배열에 저장됨


```cpp
  // Pull in only the operation implementations we need.
  // This relies on a complete list of all the ops needed by this graph.
  // An easier approach is to just use the AllOpsResolver, but this will
  // incur some penalty in code space for op implementations that are not
  // needed by this graph.
  //
  // tflite::ops::micro::AllOpsResolver resolver;
  // NOLINTNEXTLINE(runtime-global-variables)
  static tflite::MicroOpResolver<3> micro_op_resolver;
  micro_op_resolver.AddBuiltin(
      tflite::BuiltinOperator_DEPTHWISE_CONV_2D,
      tflite::ops::micro::Register_DEPTHWISE_CONV_2D());
  micro_op_resolver.AddBuiltin(tflite::BuiltinOperator_CONV_2D,
                               tflite::ops::micro::Register_CONV_2D());
  micro_op_resolver.AddBuiltin(tflite::BuiltinOperator_AVERAGE_POOL_2D,
                               tflite::ops::micro::Register_AVERAGE_POOL_2D());
```

- 모델 실행에 필요한 연산을 로드하기 위해 MicroOpResolver 객체 생성


```cpp
  // Build an interpreter to run the model with.
  static tflite::MicroInterpreter static_interpreter(
      model, micro_op_resolver, tensor_arena, kTensorArenaSize, error_reporter);
  interpreter = &static_interpreter;

  // Allocate memory from the tensor_arena for the model's tensors.
  TfLiteStatus allocate_status = interpreter->AllocateTensors();
  if (allocate_status != kTfLiteOk) {
    TF_LITE_REPORT_ERROR(error_reporter, "AllocateTensors() failed");
    return;
  }
```

- MicroInterpreter 객체 생성

- 인터프리터에서 모델 실행을 위해 사용할 텐서 메모리 할당


```cpp
  // Get information about the memory area to use for the model's input.
  input = interpreter->input(0);
}
```

- 입력 텐서의 포인터를 input 변수에 저장

#### loop() 함수

```cpp
// The name of this function is important for Arduino compatibility.
void loop() {
  // Get image from provider.
  if (kTfLiteOk != GetImage(error_reporter, kNumCols, kNumRows, kNumChannels,
                            input->data.uint8)) {
    TF_LITE_REPORT_ERROR(error_reporter, "Image capture failed.");
  }
```

- 이미지 추출기 GetImage() 함수를 호출하여 입력 텐서에 이미지 데이터 저장

```cpp
  // Run the model on this input and make sure it succeeds.
  if (kTfLiteOk != interpreter->Invoke()) {
    TF_LITE_REPORT_ERROR(error_reporter, "Invoke failed.");
  }
```

- 인터프리터를 실행하여 모델의 출력 결과를 생성

```cpp
  TfLiteTensor* output = interpreter->output(0);

  // Process the inference results.
  uint8_t person_score = output->data.uint8[kPersonIndex];
  uint8_t no_person_score = output->data.uint8[kNotAPersonIndex];
  RespondToDetection(error_reporter, person_score, no_person_score);
}
```

- 출력 텐서 데이터에서 '사람 있음' 점수와 '사람 없음' 점수를 읽음

- 감지 응답기 RespondToDetection() 함수를 호출하여 결과 처리

#### 애플리케이션 실행
- 애플리케이션 빌드 수행

In [ ]:
! make -f tensorflow/lite/micro/tools/make/Makefile person_detection

- 빌드된 애플리케이션 실행 파일 위치 (colab에서 빌드한 경우)
  - tensorflow/lite/micro/tools/make/gen/linux_x86_64/bin/person_detection

- 출력 예제


person score:129 no person score 202


person score:129 no person score 202


person score:129 no person score 202



In [9]:
! tensorflow/lite/micro/tools/make/gen/linux_x86_64/bin/person_detection

person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
person score:129 no person score 202
p

## 8.5 마이크로컨트롤러에 배포하기

- 장치별 버전
  - 카메라 장치에 맞게 image_provider.cc를 별도로 구현해야 함
  - 출력도 마찬가지므로 detection_responder.cc도 장치별 버전이 필요함

- 아두이노용 코드
  - arduino_main.cpp
  - arduino_detection_responder.cpp
  - arduino_image_provider.cpp
  - 아두이노 IDE에서 person_detection 예제를 열었을 때 보이는 person_detection 코드가 main_functions.cc에 해당




- 현재 실습에 사용하는 장치
  - Arduino TinyML Shield에 장착하여 사용하는 OV7675 camera
  - 여기에 맞게 제작된 예제 코드를 이용할 것임

- Arduino IDE에서 Harvard_TinyMLx 라이브러리
  - Arduino IDE 툴 > 라이브러리 관리 메뉴 선택 후 라이브러리 매니저 창에서 Harvard_TinyMLx 검색하여 설치

- Arduino IDE 파일 > 예제 > Harvard_TinyMLx > person_detection 선택하여 예제 열기
  - 이 예제의 arduino_image_provider.cpp 코드에서 실습 장치에 맞는 GetImage() 함수 구현


#### 아두이노 이미지 추출기 (arduino_image_provider.cpp)
- 카메라 장치에서 이미지를 캡처하여 96x96 크기로 만든 후 데이터 배열에 저장

```cpp
/* Copyright 2019 The TensorFlow Authors. All Rights Reserved.
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at
  http://www.apache.org/licenses/LICENSE-2.0
  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License.
  ==============================================================================*/

#include "image_provider.h"

#ifndef ARDUINO_EXCLUDE_CODE

#include "Arduino.h"
#include <TinyMLShield.h>

// Get an image from the camera module
TfLiteStatus GetImage(tflite::ErrorReporter* error_reporter, int image_width,
                      int image_height, int channels, int8_t* image_data) {

  byte data[176 * 144]; // Receiving QCIF grayscale from camera = 176 * 144 * 1

  static bool g_is_camera_initialized = false;
  static bool serial_is_initialized = false;

  // Initialize camera if necessary
  if (!g_is_camera_initialized) {
    if (!Camera.begin(QCIF, GRAYSCALE, 5, OV7675)) {
      TF_LITE_REPORT_ERROR(error_reporter, "Failed to initialize camera!");
      return kTfLiteError;
    }
    g_is_camera_initialized = true;
  }

  // Read camera data
  Camera.readFrame(data);

  int min_x = (176 - 96) / 2;
  int min_y = (144 - 96) / 2;
  int index = 0;

  // Crop 96x96 image. This lowers FOV, ideally we would downsample but this is simpler.
  for (int y = min_y; y < min_y + 96; y++) {
    for (int x = min_x; x < min_x + 96; x++) {
      image_data[index++] = static_cast<int8_t>(data[(y * 176) + x] - 128); // convert TF input image to signed 8-bit
    }
  }

  return kTfLiteOk;
}

#endif  // ARDUINO_EXCLUDE_CODE
```




- 초기화
  - 해상도 QCIF(176x144) 설정
  - Grayscale 설정
  - 카메라 OV7675 설정

- OV7675 카메라로부터 이미지를 캡처
  - Camera.readFrame(data);

- 96x96 사이즈 이미지에 해당하는 부분을 image_data 배열에 저장
  - 8 비트 int로 변환

#### 아두이노 감지 응답기 (arduino_detection_responder.cpp)
- 추론이 실행될 때마다 파란색 LED를 깜박임
- 사람이 감지되면 녹색 LED를 켜고 사람이 감지되지 않으면 빨간색 LED를 켬



```cpp
#if defined(ARDUINO) && !defined(ARDUINO_ARDUINO_NANO33BLE)
#define ARDUINO_EXCLUDE_CODE
#endif  // defined(ARDUINO) && !defined(ARDUINO_ARDUINO_NANO33BLE)

#ifndef ARDUINO_EXCLUDE_CODE

#include "detection_responder.h"

#include "Arduino.h"

// Flash the blue LED after each inference
void RespondToDetection(tflite::ErrorReporter* error_reporter,
                        int8_t person_score, int8_t no_person_score) {
  static bool is_initialized = false;
  if (!is_initialized) {
    // Pins for the built-in RGB LEDs on the Arduino Nano 33 BLE Sense
    pinMode(LEDR, OUTPUT);
    pinMode(LEDG, OUTPUT);
    pinMode(LEDB, OUTPUT);
    is_initialized = true;
  }

  // Note: The RGB LEDs on the Arduino Nano 33 BLE
  // Sense are on when the pin is LOW, off when HIGH.

  // Switch the person/not person LEDs off
  digitalWrite(LEDG, HIGH);
  digitalWrite(LEDR, HIGH);

  // Flash the blue LED after every inference.
  digitalWrite(LEDB, LOW);
  delay(100);
  digitalWrite(LEDB, HIGH);

  // Switch on the green LED when a person is detected,
  // the red when no person is detected
  if (person_score > no_person_score) {
    digitalWrite(LEDG, LOW);
    digitalWrite(LEDR, HIGH);
  } else {
    digitalWrite(LEDG, HIGH);
    digitalWrite(LEDR, LOW);
  }

  TF_LITE_REPORT_ERROR(error_reporter, "Person score: %d No person score: %d",
                       person_score, no_person_score);
}

#endif  // ARDUINO_EXCLUDE_CODE
```

- 초기화
  - RGB LED를 출력용으로 설정

- 녹색, 빨간색 LED를 끔 (HIGH: off, LOW: on 의미)

- 파란색 LED를 켰다가 잠시 후에 끔

- 사람이 감지되면 녹색 LED를 켜고, 사람이 감지되지 않으면 빨간색 LED를 켬
  - 감지 여부는 '사람 있음' 점수(person_score)와 '사람 없음' 점수(no_person_score)를 비교하여 결정

- 마지막으로 error_reporter 객체를 이용하여 점수를 시리얼 포트에 출력